In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import operator
from openml import datasets, tasks, flows, config

import itertools

def powerset(L):
    pset = set()
    for n in xrange(len(L) + 1):
        for sset in itertools.combinations(L, n):
            pset.add(sset)
    return pset


In [2]:
class Edge:
    def __init__(self, source, dest, f, t):
        self.source = source
        self.dest = dest
        self.f = f
        self.t = t
graph = {'edges':
             [Edge(0,1,3,0), Edge(0, 2, 3, 0), Edge(1,3,3,25), 
             Edge(3,4,3,10), Edge(2,5, 2,25), Edge(2,4, 3,10),
             Edge(4,6,1,60), Edge(4,7, 2,30)],
        'vertices':
             {0:10, 1:8, 2:2, 3:40, 4:30, 5:1, 6:2, 7: 3}}

In [3]:
def o_neighbors(graph, v):
    n = []
    for e in graph['edges']:
        if e.source == v:
            n = n + [e.dest]
    return n
def get_edge(graph, v0, v1):
    for e in graph['edges']:
        if e.source == v0 and e.dest == v1:
            return e
    return None

def path(graph, v0, v1):
    def find_path(graph, v0, v1, path = []):
        path = path + [v0]
        active = o_neighbors(graph,v0)
        if v0 == v1:
            return [path]
        paths  = []
        for v in active:
            if v not in path:
                newpaths = find_path(graph, v, v1, path)
                for newpath in newpaths:
                    paths.append(newpath)

        return paths
    all_paths = find_path(graph, v0, v1)
    return set().union(*all_paths)  
def path_edge(graph, v0, v1):
    vs = path(graph, v0, v1)
    edges = []
    for p in powerset(vs):
        if len(p) == 2:
            e = get_edge(graph, p[0], p[1])
            if e:
                edges.append(e)
    return edges
             
# for e in path_edge(graph, 0, 7):
#     print e.source, e.dest

In [4]:
def total_size(graph, mat):
    total = 0
    for k,v in graph['vertices'].iteritems():
        if k in mat:
            total = total + v
    return total

In [5]:
def recreation_cost(graph, v):
    cost = 0
    for e in path_edge(graph, 0, v):
        if e.dest != v:
            cost+= (e.t)
        else:
            alpha = e.f
            cost += (e.t)
    return alpha * cost
#recreation_cost(graph, 7)

In [6]:
def total_recreation_cost(graph, mat):
    total = 0
    for e in graph['edges']:
        if e.dest not in mat:
            total = total + (e.f * e.t)
    return total

In [7]:
#algorithm 2
def algorithm_materialization(graph, MAX_SIZE = 55):
    mat = [0]
    T = graph['vertices'][0]
    estimates = {}
    for i in range(1,8):
        estimates[i] = float(recreation_cost(graph, i)) / float(graph['vertices'][i])
    for k,v in sorted(estimates.items(), key=operator.itemgetter(1), reverse=True):
        if T + graph['vertices'][k] <= MAX_SIZE:
            mat.append(k)
            T = T + graph['vertices'][k]
    return mat
result = algorithm_materialization(graph)            
print sorted(result), total_size(graph, result), total_recreation_cost(graph, result)

[0, 1, 4, 5, 6, 7] 54 75


In [8]:
import itertools
def powerset(L):
    pset = set()
    for n in xrange(len(L) + 1):
        for sset in itertools.combinations(L, n):
            pset.add(sset)
    return pset

def brute_force(graph, MAX_SIZE = 55):
    results = {}
    for l in powerset(graph['vertices'].keys()):
        if 0 in l:
            weight = 0
            for v in l:
                weight += graph['vertices'][v]
            results[l] = (total_recreation_cost(graph, l), weight)
    feasible = {}
    for k,v in results.iteritems():
        if v[1] <= MAX_SIZE:
            feasible [k] = v
    return list(sorted(feasible.items(), key=lambda v:v[1][0])[0][0])

result = brute_force(graph)  
print sorted(result), total_size(graph, result), total_recreation_cost(graph, result)

[0, 1, 4, 5, 6, 7] 54 75


In [9]:
VERY_LARGE_IMPROVEMENT_RATIO = 10000000
# def improv_ratio(graph,mat,v,u):
#     if u == -1:
#         temp = list(mat)
#         temp.append(v)
#         return VERY_LARGE_IMPROVEMENT_RATIO * estimate_cost(graph, temp )
#     temp = list(mat)
#     r = estimate_cost(graph, temp)
#     temp.remove(u)
#     temp.append(v)
#     r_added = estimate_cost(graph, temp)
#     s = float(vertices[u])/float(vertices[v])
#     return (float(r)/float(r_added))*s
def improv_ratio(graph,mat,v,u):
    if u == -1:
        temp = list(mat)
        temp.append(v)
        return VERY_LARGE_IMPROVEMENT_RATIO * estimate_time(graph, temp )
    new_mat = list(mat)
    new_mat.remove(u)
    new_mat.append(v)
   #print vertices[v],vertices[0]
    print v,u,estimate_time(graph, mat) - estimate_time(graph, new_mat)
    return estimate_time(graph, mat) - estimate_time(graph, new_mat)


In [46]:
#algorithm 1
def algorithm_materialization(graph, MAX_SIZE = 55):
    mat = [0]
    T = graph['vertices'][0]
    estimates = {}
    for i in range(1,8):
        estimates[i] = estimate_time(graph, {0,i})
    for k,v in sorted(estimates.items(), key=operator.itemgetter(1)):
        if T + graph['vertices'][k] <= MAX_SIZE:
            mat.append(k)
            T = T + graph['vertices'][k]
    print mat, T, estimate_time(graph, mat)
    while True:
#         for v in vertices.keys():
#             if v not in mat:
#                 if T + vertices[v] <= MAX_SIZE:
#                     mat.append(v)
#                     T = T + vertices[v]
#                     print 'adding new vertex {}'.format(v)
        candidates = {}
        for v in graph['vertices'].keys():
            if v not in mat:     
                if T + graph['vertices'][v] <= MAX_SIZE:
                    candidates[(v,-1)] = improv_ratio(graph, mat, v, -1)  
                for u in mat:
                    if u != 0:
                        if T + graph['vertices'][v] - graph['vertices'][u] <= MAX_SIZE:
#                             improv_v_u_mat = improv_ratio(graph, mat, v, u)
#                             temp_mat = list(mat)
#                             temp_mat.remove(u)
#                             temp_mat.append(v)
#                             improv_u_v_mat = improv_ratio(graph, temp_mat, u, v)
#                             #print v,u,improv_v_u_mat, improv_u_v_mat
#                             if improv_v_u_mat > improv_u_v_mat:
                                candidates[(v,u)] = improv_ratio(graph, mat, v,u)
                                
                                
        if len(candidates) == 0:
            return mat,T, estimate_time(graph, mat)
        print candidates
        top_candidates = sorted(candidates.items(), key=operator.itemgetter(1), reverse = True)[0]
        v = top_candidates[0][0]
        u = top_candidates[0][1]
       #print top_candidates
        #print mat, T
        #print v,u, total_size(vertices, mat), improv_ratio(graph, mat, v,u)
        mat.append(v)
        if u != -1:
            mat.remove(u)
            T = T + graph['vertices'][v] - graph['vertices'][u] 
        else:
            T = T + graph['vertices'][v]
        
            

In [50]:
#algorithm 2
def algorithm_materializatio2(graph, MAX_SIZE = 55):
    mat = [0]
    T = graph['vertices'][0]
    estimates = {}
    for i in range(1,8):
        estimates[i] = estimate_time(graph, {0,i}) * total_size(graph, {0,i})
    print estimates
    for k,v in sorted(estimates.items(), key=operator.itemgetter(1)):
        if T + graph['vertices'][k] <= MAX_SIZE:
            mat.append(k)
            T = T + graph['vertices'][k]
    print mat, T, estimate_time(graph, mat)
    
algorithm_materializatio2(graph) 
            

{1: 5544, 2: 3696, 3: 11800, 4: 10040, 5: 2871, 6: 3012, 7: 3263}
[0, 5, 6, 7, 2, 1] 26 135


In [78]:
for i in range(1,8):
    print '{} &'.format(float(recreation_cost(graph, i)) / float(graph['vertices'][i])),
  
    

0.0 & 0.0 & 1.875 & 4.5 & 50.0 & 52.5 & 50.0 &
